## Evaluating the solution 💯

We have a working solution, but we don't know yet how often it is "correct".

We can evaluate the output quality of the LLM in different ways, using both *offline* and *online* metrics. In our current situation, before we have brought the application to production, we are in the *offline* phase.

In this notebook, we will explore different forms of evaluations in the offline phase.

Some useful resources about LLM evals are [this blogpost](https://hamel.dev/blog/posts/evals/) and [this blogpost](https://medium.com/data-science-at-microsoft/evaluating-llm-systems-metrics-challenges-and-best-practices-664ac25be7e5).

## Unit testing basic functionality

We might not always have labeled data available. In any case, unit tests can help you build confidence in the system you're shipping to production.

We can test helper functions like we're already doing in `tests/test_extraction.py`. 

But we can also test the main functions that define our application, such as our `extract_general_info`, which extracts title and summary, or `extract_businesses_involved`, which extracts the businesses involved in the news article.

For example, we can assert that the summary is no more than 2 sentences, or that the extracted businesses indeed occur in the article.

What else could we test for?

In [2]:
from llmops_training.news_reader.data import get_bbc_news_sample
from llmops_training.news_reader.extraction import (
    extract_general_info,
    extract_businesses_involved,
    get_general_info_prompt_template,
    get_businesses_involved_prompt_template,
)

In [ ]:
data = get_bbc_news_sample()
article = data[data["is_business"]].iloc[0].article

In [3]:
article = """The US firm behind the Roomba smart vacuum cleaner, iRobot, has filed for bankruptcy protection after facing competition from Chinese rivals and being hit by tariffs.
Under the so-called pre-packaged Chapter 11 process, the main manufacturer of its devices, Shenzhen-based Picea Robotics, will take ownership of the firm.
The tough commercial landscape had forced iRobot to cut its prices and make major investments in new technology, according to documents filed on Sunday.
US import duties of 46% on goods from Vietnam, where most of iRobot's devices for the American market are made, increased its costs by $23m (£17.2m) this year, the firm said.
The loss-making company was valued at $3.56bn in 2021 after the pandemic helped to drive strong demand for its products. It is now valued at around $140m.
On Friday, iRobot's shares fell by more than 13% on the technology-heavy Nasdaq trading platform in New York.
iRobot said the bankruptcy filing was not expected to disrupt its app, supply chains or product support.
Founded in 1990 by three members of the Massachusetts Institute of Technology's (MIT) Artificial Intelligence Lab, iRobot initially focused on defence and space technology before launching the Roomba in 2002.
The Roomba holds about 42% of the US market share and 65% of the Japanese market share for robotic vacuum cleaners, according to the company.
Last year, a planned $1.7bn takeover deal by online retail giant Amazon was derailed by the European Union's competition watchdog.
Trade tariffs imposed by US Donald Trump on goods entering America from overseas has added to costs to many businesses, including iRobot, which rely on imports for product manufacturing.
Trump has argued that the import taxes will boost American jobs and industry.
Picea is a manufacturer of robotic vacuum cleaners, with research and development and production facilities in China and Vietnam.
It has more than 7,000 employees worldwide and has sold more than 20 million robotic vacuum cleaners."""

> **Exercise** 📝
>
> - Fill in the TODO's in the test functions below.
> - Run the tests in the subsequent cell
> - Move these tests to the `tests/test_extraction.py` file
> - Run the tests from the command line using `uv run pytest tests`
> 


In [4]:
def test_summary_not_too_long(article: str):
    summary = extract_general_info(get_general_info_prompt_template(), article).summary
    summary_length = len(summary.split(". "))
    assert summary_length <= 2, f"Summary is too long: {summary_length} sentences"


def test_extracted_businesses_are_in_article(article: str):
    businesses = extract_businesses_involved(
        get_businesses_involved_prompt_template(), article
    ).businesses
    for business in businesses:
        # TODO: Fill me in! Assert that the business is in the article
        assert business.lower() in article.lower(), f"Business '{business}' not found in the article"

In [ ]:
# %load ../solutions/evaluating-the-solution/unit-tests.py

In [ ]:
test_summary_not_too_long(article)
test_extracted_businesses_are_in_article(article)

print("🎉 Tests passed!")

## Evaluating using labeled data

To evaluate your application in more depth in a structured manner, it is essential to collect labeled data for your use case.

We can then define metrics, such as accuracy or ROUGE score, that we can iterate and improve upon.

Luckily, we already have some labeled examples that we can use for evaluating. We only load a few in this training, so the evaluations remain cheap and go relatively quick.

In [ ]:
from typing import List, Optional

import pandas as pd

from llmops_training.news_reader.data import get_evaluation_data
from llmops_training.news_reader.extraction import (
    GeneralInfo, 
    extract_business_category_from_articles,
    get_business_category_prompt_template
)

In [ ]:
# A few articles for illustration
eval_data = get_evaluation_data()
eval_data

We can evaluate different aspects of our application, such as:
- How many articles are correctly classified as *business* news or not
- How many of the titles are correctly extracted
- How many times the pipeline succeeds or errors
- How well the summaries corresponds to the labeled summaries

Let's implement the first one!

First, we classify for each article if it is business news or not. We can then compare this to the labeled data:

In [ ]:
business_category_list = extract_business_category_from_articles(
    get_business_category_prompt_template(),
    eval_data["article"].tolist()
)
business_category_list

> **Exercise** 📝
>
> - Inspect the `evaluate_business_classification` function below.
> - Fill in the TODO's in the function, so it accurately computes the accuracy of the business classification.
> - Inspect the `evaluation` module in our package, which already has some evaluation functions implemented.
> - Move this function to the `evaluation` module in our package.
> - Uncomment the TODO's in the `evaluation` module to include this evaluation function.
> - Run the `evaluation` module as script from the command line using:
>   ```bash
>   uv run python -m llmops_training.news_reader.evaluation
>   ```
> - Can you see the `business_classification_accuracy` in the output? 🎉

In [ ]:
def evaluate_business_classification(
    business_category_list: List[Optional[GeneralInfo]], data: pd.DataFrame
) -> float:
    """Return accuracy of classification of whether an article is about business.

    Data should contain:
    - a column "is_business" with a boolean indicating whether the article is about business.

    The provided list of business categories should be in the same order as the data.
    """

    correct = 0
    incorrect = 0
    for i, row in data.iterrows():
        if business_category_list[i] is None:
            continue

        is_about_business_prediction = business_category_list[i].is_about_business
        is_about_business_labeled = row["is_business"]
        
        incorrect += 1  # TODO: Extend me! Write if-else statement to increment correct or incorrect

    accuracy = correct / (correct + incorrect)
    return accuracy

In [ ]:
# %load ../solutions/evaluating-the-solution/evaluate-business-classification.py

## (Bonus) Making evaluations part of tests

We can also include these evaluations in our test suite. For example, we can assert that the business classification accuracy is above a certain threshold. We can do so similarly for each metric.

> **Exercise** 📝
>
> - Inspect the `test_run_evaluation` function below.
> - Fill in the TODO's in the function, so it asserts that the metrics pass rates are satisfied.
> - Move this function to the `tests/test_evaluation.py` file.
> - Run the tests from the command line using `uv run pytest tests`
>
> 💡 Hint:
>
> - Choose sensible pass rates for the metrics. Don't be too strict for now, or your tests will fail quite often.
> - From the summarization metrics, focus just on `summarization_rouge_1`

In [ ]:
from llmops_training.news_reader.evaluation import run_evaluation

In [ ]:
def test_run_evaluation(evaluation_data: pd.DataFrame):
    results = run_evaluation(evaluation_data)

    # TODO: Fill me in! Choose pass rates for metrics and assert they are satisfied

In [ ]:
# %load ../solutions/evaluating-the-solution/evaluation-threshold.py

---